In [ ]:
%matplotlib inline

from __future__ import print_function, division

import os, sys
import glob
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import ipywidgets as widgets
from ipywidgets import interact, interactive, interact_manual
from ipywidgets import FloatSlider, IntSlider, Dropdown
from ipywidgets import Checkbox, RadioButtons, fixed 

from IPython.display import display, Math, clear_output

from astropy.convolution import convolve, Gaussian1DKernel

import seaborn as sns
sns.set(context="poster", font_scale=1.5)


## import from local files

## Boilerplate path hack to give access to full clustered_SNe package
import sys, os
if __package__ is None:
    if os.pardir not in sys.path[0]:
        file_dir = os.getcwd()
        sys.path.insert(0, os.path.join(file_dir, 
                                        os.pardir, 
                                        os.pardir))

from clustered_SNe import data_dir_default
from clustered_SNe.analysis.constants import m_proton, pc, yr, M_solar, \
                                   gamma, E_0, metallicity_solar
from clustered_SNe.analysis.sedov.dimensionalize_sedov import dimensionalized_sedov
from clustered_SNe.analysis.sedov.closed_form_sedov import SedovSolution
from clustered_SNe.analysis.parse import RunSummary, Overview, cols
from clustered_SNe.analysis.visualize_helpers import single_run, \
                                                     conduction_comparisons, \
                                                     SNe_distplot, \
                                                     plot_zones, \
                                                     plot_shock_location, \
                                                     plot_energy, \
                                                     plot_momentum, \
                                                     plot_luminosity

        
# Holds the most recent run, if more analysis is desired
run_summary = RunSummary()    

In [ ]:
data_dir = "../src/"
data_dir = data_dir_default


# you don't need the full uuid, just enough to distinguish it
id_default = ""

result_widget = interact_manual(single_run,
                                data_dir = widgets.Text(data_dir),
                                id = widgets.Text(id_default))
def update_run_summary_single_run(widget):
    """Copy the latest result into run_summary
    
    Don't reuse this function definition; needs to be bound to the correct result_widget"""
    run_summary.replace_with(result_widget.widget.result)
result_widget.widget.children[0].on_submit(update_run_summary_single_run)
result_widget.widget.children[1].on_submit(update_run_summary_single_run)
result_widget.widget.children[2].on_click( update_run_summary_single_run)



In [ ]:
print(run_summary.first_unreasonable_energy())
print(run_summary.is_time_resolved())
print(run_summary.is_converged())

In [ ]:
np.argmax(run_summary.momentum)

In [ ]:
len(run_summary.momentum)

In [ ]:
plot_zones(run_summary)

In [ ]:
t = (run_summary.times) / yr
t_offset = run_summary.times[0] / yr

from scipy import stats
SNe_times = run_summary.overview.SNe_times / yr
sns.distplot(SNe_times)
# plt.plot(SNe_times - t_offset, 4e-8 * (SNe_times/1e7)**-.63)
# plt.plot(SNe_times, )

shape,loc,scale = stats.powerlaw.fit(SNe_times, floc=t_offset, fscale=SNe_times.max()- t_offset)
print(shape,loc,scale)
plt.plot(SNe_times, stats.powerlaw.pdf(SNe_times, shape,loc=loc, scale=scale), label="fit")
plt.legend(loc="best")

a_1 = shape-1
a_2 = (a_1 + 3)/5
print("a_1 (dE/dt ~ t^{a_1}) = ", a_1)
print("a_2 (R ~ t^{a_2}) = ", a_2)

plt.figure()

R = run_summary.R_shock / pc
t = (run_summary.times) / yr
t_offset = run_summary.times[0] / yr
mask = t > .5
t_0 = 1e7
R_0 = R[np.argmin(np.abs((t-t_offset) - t_0))]

plt.plot(t[mask], R[mask], label="data")
plt.plot(t[mask], R_0 * ((t[mask] - t_offset)/t_0)**.6,  linestyle="dashed", label="predicted (a_1 = 0)")
plt.plot(t[mask], R_0 * ((t[mask] - t_offset)/t_0)**a_2, linestyle="dashed", label="predicted (a_1 from SNe)")
plt.plot(t[mask], R_0 * ((t[mask] - t_offset)/t_0)**.53, linestyle="dashed", label="fit by eye")
plt.xlabel("time [yr]")
plt.ylabel("R [pc]")
plt.legend(loc="best")
# plt.xscale("log")
# plt.yscale("log")

plt.figure()
momentum = run_summary.momentum
t_0 = 1e7
momentum_0 = momentum[np.argmin(np.abs((t-t_offset) - t_0))]

plt.plot(t[mask], momentum[mask], label="data")
plt.plot(t[mask], momentum_0 * ((t[mask] - t_offset)/t_0)**1.4, linestyle="dashed", label="predicted (a_1 = 0)")
plt.plot(t[mask], momentum_0 * ((t[mask] - t_offset)/t_0)**(4*a_2 - 1), linestyle="dashed", label="predicted (a_1 free)")
plt.plot(t[mask], momentum_0 * ((t[mask] - t_offset)/t_0)**((.53*4) - 1), linestyle="dashed", label="predicted by shock fit (a_1=0)")
plt.plot(t[mask], momentum_0 * ((t[mask] - t_offset)/t_0)**1.1, linestyle="dashed", label="fit by eye")
plt.xlabel("time [yr]")
plt.ylabel("momentum [g cm s-1]")
plt.legend(loc="best")
# plt.xscale("log")
# plt.yscale("log")

In [ ]:
plot_shock_location(run_summary)

In [ ]:
_ = interact(plot_energy,
         run_summary=fixed(run_summary),
         x_axis = RadioButtons(options=["time", "checkpoints"]))

In [ ]:
_ = interact(plot_momentum,
         run_summary=fixed(run_summary),
         x_axis = RadioButtons(options=["time", "checkpoints"]),
         clear_previous=fixed(True),
         distplot=fixed(True),
         y_axis_scaling=fixed("mass"))
# plt.ylim(ymax=5e3)
# plt.ylim(ymin=-5e3)

In [ ]:
_ = interact(plot_luminosity,
         run_summary=fixed(run_summary),
         x_axis = RadioButtons(options=["time", "checkpoints"]))
